In [1]:
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
from osprey.actions.reader import reader_nemo_field

In [4]:
field = reader_nemo_field(expname='pi00', startyear=1990, endyear=1999, varname='thetao')

2024-12-18 14:24:12,068 - INFO - Data available in the range 1990-1999.


In [14]:
field

<xarray.DataArray 'thetao' (time: 120, z: 31, y: 148, x: 180)> Size: 396MB
dask.array<concatenate, shape=(120, 31, 148, 180), dtype=float32, chunksize=(12, 31, 148, 180), chunktype=numpy.ndarray>
Coordinates:
    lat      (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
    lon      (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
  * z        (z) float32 124B 5.0 15.0 25.0 35.01 ... 4.25e+03 4.75e+03 5.25e+03
  * time     (time) object 960B 1990-01-16 12:00:00 ... 1999-12-16 12:00:00
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           temperature
    units:               degC
    online_operation:    average
    interval_operation:  5400 s
    interval_write:      1 month
    cell_methods:        time: mean (interval: 5400 s)

In [5]:
from osprey.means.means import timemean

In [8]:
ave = timemean(data=field, format='monthly', use_cftime=False)
ave

<xarray.DataArray 'thetao' (month: 12, z: 31, y: 148, x: 180)> Size: 40MB
dask.array<stack, shape=(12, 31, 148, 180), dtype=float32, chunksize=(1, 31, 148, 180), chunktype=numpy.ndarray>
Coordinates:
    lat      (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
    lon      (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
  * z        (z) float32 124B 5.0 15.0 25.0 35.01 ... 4.25e+03 4.75e+03 5.25e+03
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           temperature
    units:               degC
    online_operation:    average
    interval_operation:  5400 s
    interval_write:      1 month
    cell_methods:        time: mean (interval: 5400 s)

In [10]:
import cftime

In [13]:
ave = field.groupby('time.month').mean(dim='time')
last_year = field['time.year'].values[-1]
dates = [cftime.DatetimeGregorian(last_year, month, 1, 0, 0, 0, has_year_zero=False) for month in range(1, 13)]
ave = xr.DataArray(data=ave, dims=["time"] + [dim for dim in ave.dims if dim != "month"], coords={"time": dates, **{dim: ave[dim] for dim in ave.dims if dim != "month"}})
ave

<xarray.DataArray 'thetao' (time: 12, z: 31, y: 148, x: 180)> Size: 40MB
dask.array<stack, shape=(12, 31, 148, 180), dtype=float32, chunksize=(1, 31, 148, 180), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 96B 1999-01-01 00:00:00 ... 1999-12-01 00:00:00
  * z        (z) float32 124B 5.0 15.0 25.0 35.01 ... 4.25e+03 4.75e+03 5.25e+03
  * y        (y) int64 1kB 0 1 2 3 4 5 6 7 8 ... 140 141 142 143 144 145 146 147
  * x        (x) int64 1kB 0 1 2 3 4 5 6 7 8 ... 172 173 174 175 176 177 178 179
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           temperature
    units:               degC
    online_operation:    average
    interval_operation:  5400 s
    interval_write:      1 month
    cell_methods:        time: mean (interval: 5400 s)

In [22]:
ave = field.groupby('time.month').mean(dim='time')
last_year = field['time.year'].values[-1]
dates = [cftime.DatetimeGregorian(last_year, month, 1, 0, 0, 0, has_year_zero=False) for month in range(1, 13)]
coords = {"time": dates}
for dim in field.coords:
    if dim not in coords:  # Non aggiungere 'time' o 'month'
        coords[dim] = field[dim]  # Mantieni le coordinate originali
ave = xr.DataArray(
    data=ave, 
    dims=["time"] + [dim for dim in ave.dims if dim != "month"],  # "time" sostituisce "month"
    coords=coords  # Usa le coordinate originali più le nuove date per "time"
)
ave

<xarray.DataArray 'thetao' (time: 12, z: 31, y: 148, x: 180)> Size: 40MB
dask.array<stack, shape=(12, 31, 148, 180), dtype=float32, chunksize=(1, 31, 148, 180), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 96B 1999-01-01 00:00:00 ... 1999-12-01 00:00:00
    lat      (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
    lon      (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
  * z        (z) float32 124B 5.0 15.0 25.0 35.01 ... 4.25e+03 4.75e+03 5.25e+03
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           temperature
    units:               degC
    online_operation:    average
    interval_operation:  5400 s
    interval_write:      1 month
    cell_methods:        time: mean (interval: 5400 s)

In [18]:
dates = [cftime.DatetimeGregorian(last_year, month, 1, 0, 0, 0, has_year_zero=False) for month in range(1, 13)]
coords = {"time": dates}
for dim in field.coords:
    if dim not in coords:  # Non aggiungere 'time' o 'month'
        coords[dim] = field[dim]  # Mantieni le coordinate originali

In [20]:
coords

{'time': [cftime.DatetimeGregorian(1999, 1, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 2, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 3, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 4, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 5, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 6, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 7, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 8, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 9, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 10, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 11, 1, 0, 0, 0, 0, has_year_zero=False),
  cftime.DatetimeGregorian(1999, 12, 1, 0, 0, 0, 0, has_year_zero=False)],
 'lat': <xarray.DataArray 'lat' (y: 148, x: 180)> Size: 107kB
 dask.array<where, shape=(148, 180), dtype=float32